In [1]:
import os
from pathlib import Path
import sys
import os
import _pickle as pickle
import json
import copy
import language_check
import re

In [10]:
preds = []

In [11]:
with open('./raw/predictionsRet.json') as json_file:
    sessions = json.load(json_file)

In [12]:
sessions[0][0]['knowledge']

['Blue Blue is one of the three primary colours of pigments in painting and traditional colour theory, as well as in the RGB colour model.',
 'Blue It lies between violet and green on the spectrum of visible light.',
 'Blue The eye perceives blue when observing light with a dominant wavelength between approximately 450 and 495 nanometres.',
 'Blue Most blues contain a slight mixture of other colors; azure contains some green, while ultramarine contains some violet.',
 'Blue The clear daytime sky and the deep sea appear blue because of an optical effect known as Rayleigh scattering.',
 'Blue An optical effect called Tyndall scattering explains blue eyes.',
 'Blue Distant objects appear more blue because of another optical effect called atmospheric perspective.',
 'Blue Blue has been an important colour in art and decoration since ancient times.',
 'Blue The semi-precious stone lapis lazuli was used in ancient Egypt for jewellery and ornament and later, in The Renaissance, to make the pi

In [13]:
ind = 0
for session in sessions:
    conv = {}
    firstDiag = session[0]['wizard_of_wikipedia']
    if(len(firstDiag.split("\n")) < 2):
        continue
    topic = firstDiag.split("\n")[0]
    conv['index'] = ind
    ind += 1
    
    conv['topic'] = topic
    conv['conversation'] = session
    
    
    preds.append(conv)

In [14]:
with open('predictionsRetEdit.json', 'w') as outfile:
    json.dump(preds, outfile)

In [7]:
preds = []
with open('./raw/predictionsGen.json') as json_file:
    sessions = json.load(json_file)

In [8]:
ind = 0
for session in sessions:
    conv = {}
    firstDiag = session[0]['WizTeacher']
    if(len(firstDiag.split("\n")) < 2):
        continue
    conv['index'] = ind
    ind += 1
    
    conv['conversation'] = session
    
    preds.append(conv)

In [9]:
with open('predictionsGenEdit.json', 'w') as outfile:
    json.dump(preds, outfile)

In [21]:
with open('./selected/ret/threeContext.json') as json_file:
    ret3 = json.load(json_file)

    
with open('./selected/gen/threeContext.json') as json_file:
    gen3 = json.load(json_file)
    
    
finalGen3 = []

In [22]:
for r in ret3:
    ind = r['origIndex']
    obj = next((x for x in gen3 if x['origIndex'] == ind), None)
    finalGen3.append(obj)


In [23]:
with open('./selected/gen/threeContext.json', 'w') as outfile:
    json.dump(finalGen3, outfile)

In [9]:
with open('./clean/ret/cleanPredictionsRet3.json') as json_file:
    sessions = json.load(json_file)

In [10]:
with open('./selected/ret/threeContext.json') as json_file:
    ret = json.load(json_file)

In [11]:
AIret = []
for r in ret:
    ind = r['origIndex']
    obj = next((x for x in sessions if x['origIndex'] == ind), None)
    candidates = [x for x in obj['candidates'] if x.strip() != r['actualResponse'].strip()][:5]
    r['suggestions'] = candidates
    
    AIret.append(r)
    
with open('./selected/ret_AI/threeContext.json', 'w') as outfile:
    json.dump(AIret, outfile)

In [13]:

for root, dirs, files in os.walk("./selected/ret_AI"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                index = 0
                try:
                    data = json.load(json_file)
                    
                    for d in data:
                        d['index'] = index
                        index += 1
                        
                        d['suggestions'] = d['suggestions'][:5]

                        del d['origIndex']
                    
                    filePath = './final/ret_AI/' + os.path.basename(fileName)
                    with open(filePath, 'w') as outfile:
                        json.dump(data, outfile)
                
                except Exception as e: print(e)

In [25]:

for root, dirs, files in os.walk("./selected/gen"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                index = 0
                try:
                    data = json.load(json_file)
                    
                    for d in data:
                        d['index'] = index
                        index += 1

                        del d['origIndex']
                    
                    filePath = './final/gen/' + os.path.basename(fileName)
                    with open(filePath, 'w') as outfile:
                        json.dump(data, outfile)
                
                except Exception as e: print(e)

In [14]:
tool = language_check.LanguageTool('en-US')
fix_spaces = re.compile(r'\s*([?!.,]+(?:\s+[?!.,]+)*)\s*')

regex = '\\"\s+([^"]+)\s+\\"'

In [40]:
text ="i love the song \" the song \" the song \" the voice \" ."




In [41]:
text = text.replace('\"',"'")
text

"i love the song ' the song ' the song ' the voice ' ."

In [42]:
regex = '\\"\s+([^"]+)\s+\\"'


subst = ""

# You can manually specify the number of replacements by changing the 4th argument
text = re.sub(regex, '\"'+r'\1'+'\"' , text)

matches = tool.check(text)
text = language_check.correct(text, matches)
text = fix_spaces.sub(lambda x: "{} ".format(x.group(1).replace(" ", "")), text).strip()

text

"I love the song ' the song ' the song ' the voice '."

In [29]:
for root, dirs, files in os.walk("./final/gen"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                
                try:
                    data = json.load(json_file)
                    
                    for d in data:
                        text = d['actualResponse']
                        
                        subst = ""
                        text = re.sub(regex, '\"'+r'\1'+'\"' , text)
                        
                        matches = tool.check(text)
                        text = language_check.correct(text, matches)
                        text = fix_spaces.sub(lambda x: "{} ".format(x.group(1).replace(" ", "")), text).strip()
                        text = text.replace("“", "'")
                        text = text.replace("”", "'")
                        text = text.replace(" ' ", "'")
                        
                        d['actualResponse'] = text
                        
                        newSuggs = []
                        
                        text = d['suggestions'][0]
                        
                        subst = ""
                        text = re.sub(regex, '\"'+r'\1'+'\"' , text)
                        
                        matches = tool.check(text)
                        text = language_check.correct(text, matches)
                        text = fix_spaces.sub(lambda x: "{} ".format(x.group(1).replace(" ", "")), text).strip()
                        text = text.replace("“", "'")
                        text = text.replace("”", "'")
                        text = text.replace(" ' ", "'")
                        
                        newSuggs.append(text)
                            
                    
                        d['suggestions'] = newSuggs
                    
                    filePath = './final/gen/' + os.path.basename(fileName)
                    with open(filePath, 'w') as outfile:
                        json.dump(data, outfile)
                
                except Exception as e: print(e)

In [17]:
for root, dirs, files in os.walk("./final/ret_AI"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                
                try:
                    data = json.load(json_file)
                    
                    for d in data:
                        text = d['actualResponse']
                        
                        subst = ""
                        text = re.sub(regex, '\"'+r'\1'+'\"' , text)
                        
                        matches = tool.check(text)
                        text = language_check.correct(text, matches)
                        text = fix_spaces.sub(lambda x: "{} ".format(x.group(1).replace(" ", "")), text).strip()
                        text = text.replace("“", "'")
                        text = text.replace("”", "'")
                        text = text.replace(" ' ", "'")
                        
                        d['actualResponse'] = text
                        
                        newSuggs = []
                        
                        for text in d['suggestions']:
                        
                            
                            subst = ""
                            text = re.sub(regex, '\"'+r'\1'+'\"' , text)
                        
                            matches = tool.check(text)
                            text = language_check.correct(text, matches)
                            text = fix_spaces.sub(lambda x: "{} ".format(x.group(1).replace(" ", "")), text).strip()
                            text = text.replace("“", "'")
                            text = text.replace("”", "'")
                            text = text.replace(" ' ", "'")
                            
                            newSuggs.append(text)
                            
                    
                        d['suggestions'] = newSuggs
                    
                    filePath = './final/ret_AI/' + os.path.basename(fileName)
                    with open(filePath, 'w') as outfile:
                        json.dump(data, outfile)
                
                except Exception as e: print(e)

In [28]:
with open('./final/ret_AI/threeContext.json') as json_file1:
    data1 = json.load(json_file1)
with open('./final/ret/threeContext.json') as json_file2:
    data2 = json.load(json_file2)
with open('./final/gen/threeContext.json') as json_file3:
    data3 = json.load(json_file3)    

In [29]:
for i in range(len(data1)):
    
    if(data1[i]['actualResponse'] != data2[i]['actualResponse'] or data1[i]['actualResponse'] != data3[i]['actualResponse']
      or data2[i]['actualResponse'] != data3[i]['actualResponse']):
        print(data1[i]['index'])